# Sky Almanac

This notebook contains some useful tools for planning the observations.  
It calculates the sunrise/sunset and the end/beginning of each twilight based on calculations using the coordinates of Rubin and AstroPy.  
It also plots the elevation and airmass for a series of designated targets!

## Import Libraries

These are the libraries that we will need to run this notebook.

In [ ]:
from astropy import coordinates as coord
from astropy import units as u
from astropy.time import Time, TimezoneInfo 
from datetime import datetime

import cycler
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import pytz

## Set Up

Here are the main configuration variables that propagate accross all the functions.

In [ ]:
# Vera Rubin Coordinates from https://www.lsst.org/scientists/keynumbers
rubin_obs = coord.EarthLocation(
    lat='-30:14:40.68', lon='-70:44:57.90', height=2647 * u.m)

tz = TimezoneInfo(utc_offset=-3*u.hour)

Declare targets for which the sky coordinates will be calculated and the date. 

In [ ]:
targets = ['HD 14943', 'HD 185975', 'HD 031128', 'HD 185975', 'HD 074000', 
           'HD 185975']

_date = "20220216"  # Beginning of the night

## Helper Functions

These are functions used later in the notebook

In [ ]:
def get_alt(t, location, time):
    """
    Get the Alt (Elevation) for a given target at a given location and at a 
    given time.
    
    Parameters
    ----------
    t : string
        Target name to be queried via Simbad.
    location : astropy.coordinates.EarthLocation
        Location of the observatory.
    time : astropy.time.Time
        Time of the observation 
        
    Returns
    -------
    float : the airmass of the object in degrees.
    """
    c = coord.SkyCoord.from_name(t)
    frame = coord.AltAz(obstime=time, location=location)
    altaz = c.transform_to(frame)
    return altaz.alt.value


def get_airmass(t, location, time):
    """
    Get the airmass for a given target at a given location and at a given time 
    using Plane Parallel Atmosphere Model.
    
    Parameters
    ----------
    t : string
        Target name to be queried via Simbad.
    location : astropy.coordinates.EarthLocation
        Location of the observatory.
    time : astropy.time.Time
        Time of the observation 
        
    Returns
    -------
    float : the airmass of the object
    """
    alt = get_alt(t, location, time)
    z = 90 - alt
    airmass = np.ma.masked_greater(1./np.cos(np.deg2rad(z)), 5.0)
    return airmass


def get_sun_elevation(t, site=rubin_obs):
    """
    Gets the Sun elevation at a given site and at a given time.
    
    Parameters
    ----------
    t : datetime
        Time in UTC
    site : astropy.coordinates.EarthLocation
        Location of the observation
    
    Returns
    -------
    float : the Sun elevation in Degrees.
    """
    sun_coords = coord.get_sun(t)
    sun_coords.location = rubin_obs
    return sun_coords.altaz.alt.value


def get_sun_when_at_angle(elevations, angle, setting=True):
    """
    Get the index of the `elevations` array that is the closest to a given 
    angle depending if the Sun is rising or setting.
    
    Parameters
    ----------
    elevations : numpy.array
        An array containing the Sun elevation.
    angle : float
        The requested Sun position.
    setting : bool
        True if the Sun is setting or False if it is Rising.
    
    Returns
    -------
    int : index associated with the requested sun position.
    """
    angle_difference = np.diff(elevations)
    angle_difference = np.insert(angle_difference, 0, 0)
    
    if setting:
        _elevations = np.ma.masked_where(angle_difference >= 0, elevations)
    else:
        _elevations = np.ma.masked_where(angle_difference <= 0, elevations)
        
    return np.ma.argmin(np.ma.abs(_elevations - angle))


def get_moon_elevation(t, site=rubin_obs):
    """
    Gets the Moon elevation at a given site and at a given time.
    
    Parameters
    ----------
    t : datetime
        Time in UTC
    site : astropy.coordinates.EarthLocation
        Location of the observation
    
    Returns
    -------
    float : the Sun elevation in Degrees.
    """
    moon_coords = coord.get_moon(t)
    moon_coords.location = rubin_obs
    return moon_coords.altaz.alt.value


def get_moon_airmass(t, site=rubin_obs):
    """
    Get the airmass of the Moon for a given target at a given location and at a 
    given time.
    
    Parameters
    ----------
    t : datetime
        Time in UTC
    site : astropy.coordinates.EarthLocation
        Location of the observation
        
    Returns
    -------
    float : the airmass of the object
    """
    moon_elevation = get_moon_elevation(t, site)
    z = 90 - moon_elevation
    moon_airmass = np.ma.masked_greater(1./np.cos(np.deg2rad(z)), 5.0)
    return moon_airmass

## Running Calculations

This is where all the magic happens.  
First, we create an array with the time range. 

In [ ]:
times = Time.now() + np.linspace(-10, 10, 300)*u.hour

Then we get the airmasses/elevations for each target at different times. 
We also get the Sun and the Moon positions.

In [ ]:
airmasses = {t: get_airmass(t, rubin_obs, times) for t in targets}
alts = {t: get_alt(t, rubin_obs, times) for t in targets}

sun_elevation = get_sun_elevation(times, site=rubin_obs)
moon_elevation = get_moon_elevation(times, site=rubin_obs)
moon_airmass = get_moon_airmass(times, site=rubin_obs)

Here we calculate the position of the Sun at the beginning of the night.

In [ ]:
# Sun Set
sun_set = times[get_sun_when_at_angle(sun_elevation, 0)]

# Civil Twilight Afternoon
end_civil_twilight = times[get_sun_when_at_angle(sun_elevation, -6)]

# Nautical Twilight Afternoon
end_nautical_twilight = times[get_sun_when_at_angle(sun_elevation, -12)]

# Astronomical Twilight Afternoon 
end_astronomical_twilight = times[get_sun_when_at_angle(sun_elevation, -18)]

print(f"\n Afternoon"
      f"\n   Sun sets at {sun_set.to_datetime(timezone=tz)}"
      f"\n   Civil Twilight ends at {end_civil_twilight.to_datetime(timezone=tz)}"
      f"\n   Nautical Twilight ends at {end_nautical_twilight.to_datetime(timezone=tz)}"
      f"\n   Astronomical Twilight ends at {end_astronomical_twilight.to_datetime(timezone=tz)}"
      f"\n\n")

Here we calculate the position of the Sun at the end of the night.

In [ ]:
# Astronomical Twilight  
begin_astronomical_twilight = times[
    get_sun_when_at_angle(sun_elevation, -18, setting=False)]

# Nautical Twilight Afternoon
begin_nautical_twilight = times[
    get_sun_when_at_angle(sun_elevation, -12, setting=False)]

# Civil Twilight Afternoon
begin_civil_twilight = times[
    get_sun_when_at_angle(sun_elevation, -6, setting=False)]

# Sun Rise
sun_rise = times[get_sun_when_at_angle(sun_elevation, 0, setting=False)]

print(f"\n Morning"
      f"\n   Astronomical Twilight ends at {begin_astronomical_twilight.to_datetime(timezone=tz)}"
      f"\n   Nautical Twilight ends at {begin_nautical_twilight.to_datetime(timezone=tz)}"
      f"\n   Civil Twilight ends at {begin_civil_twilight.to_datetime(timezone=tz)}"
      f"\n   Sun sets at {sun_rise.to_datetime(timezone=tz)}"
      f"\n\n")

## Plotting Target Airmass vs Time

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(num="Airmass", dpi=120, figsize=(8, 3))
# ax.set_prop_cycle('color', ["darkgrey", "darkorange", "limegreen", "mediumpurple", "yellowgreen"])

for t, a in airmasses.items():
    ax.plot(times.to_datetime(timezone=tz), a, label=t)

ax.plot(times.to_datetime(timezone=tz)[::10], 
        moon_airmass[::10], 'o', markersize=10, color='gold', label='Moon')    
    
ax.axvspan(sun_set.to_datetime(timezone=tz), 
           end_civil_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.05)

ax.axvspan(end_civil_twilight.to_datetime(timezone=tz), 
           end_nautical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.15)

ax.axvspan(end_nautical_twilight.to_datetime(timezone=tz), 
           end_astronomical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.3)

ax.axvspan(end_astronomical_twilight.to_datetime(timezone=tz), 
           begin_astronomical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.5)

ax.axvspan(begin_astronomical_twilight.to_datetime(timezone=tz), 
           begin_nautical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.3)

ax.axvspan(begin_nautical_twilight.to_datetime(timezone=tz), 
           begin_civil_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.15)

ax.axvspan(begin_civil_twilight.to_datetime(timezone=tz), 
           sun_rise.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.05)

myFmt = mdates.DateFormatter("%H:%M", tz=tz)
ax.xaxis.set_major_formatter(myFmt)
ax.invert_yaxis()
ax.set_ylim(5, 0.5)
ax.set_xlim(
    (sun_set - (end_civil_twilight - sun_set)).to_datetime(timezone=tz), 
    (sun_rise + (sun_rise - begin_civil_twilight)).to_datetime(timezone=tz))

ax.set_axisbelow(False)
ax.grid(":", lw=0.25, color='k', alpha=0.25)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel("Local time (HH:MM)")
ax.set_ylabel("Airmass")

fig.tight_layout()
plt.show()

## Plotting Target Elevation vs Time 

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(num="Elevation", dpi=120, figsize=(8, 4))

for t, a in alts.items():
    ax.plot(times.to_datetime(timezone=tz), a, label=t)

ax.plot(times.to_datetime(timezone=tz)[::10], 
        moon_elevation[::10], 'o', markersize=10, color='gold', label='Moon')     
    
ax.axvspan(sun_set.to_datetime(timezone=tz), 
           end_civil_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.05)

ax.axvspan(end_civil_twilight.to_datetime(timezone=tz), 
           end_nautical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.15)

ax.axvspan(end_nautical_twilight.to_datetime(timezone=tz), 
           end_astronomical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.3)

ax.axvspan(end_astronomical_twilight.to_datetime(timezone=tz), 
           begin_astronomical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.5)

ax.axvspan(begin_astronomical_twilight.to_datetime(timezone=tz), 
           begin_nautical_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.3)

ax.axvspan(begin_nautical_twilight.to_datetime(timezone=tz), 
           begin_civil_twilight.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.15)

ax.axvspan(begin_civil_twilight.to_datetime(timezone=tz), 
           sun_rise.to_datetime(timezone=tz), 
           fc="royalblue", alpha=0.05)

ax.axvline(Time.now().to_datetime(timezone=tz), 
           color='black', lw=0.5)

myFmt = mdates.DateFormatter("%H:%M", tz=tz)
ax.xaxis.set_major_formatter(myFmt)
ax.set_axisbelow(False)
ax.grid(":", lw=0.25, color='k', alpha=0.25)
ax.legend(loc='center left', bbox_to_anchor=(1.15, 0.5))
ax.set_xlabel("Local time (HH:MM)")
ax.set_ylabel("Alt [deg]")
ax.set_ylim(15, 90)
ax.set_xlim(
    (sun_set - (end_civil_twilight - sun_set)).to_datetime(timezone=tz), 
    (sun_rise + (sun_rise - begin_civil_twilight)).to_datetime(timezone=tz))


f = lambda x: 1. / np.cos(np.deg2rad(90. - x))
y_ticks = ax.get_yticks()

ax2 = ax.twinx()
ax2.set_yticks(y_ticks)
ax2.set_ylim(15, 90)
ax2.set_yticklabels([f"{f(l):.2f}" for l in y_ticks])
ax2.set_ylabel("Airmass [--]")
fig.suptitle("Target Elevation vs Time\n"
             f"{Time.now().to_datetime(timezone=tz):%Y-%m-%d %H:%M}")

fig.tight_layout()
plt.show()